# Customer Retention Analysis: Telco Churn Dataset

### Understanding customer behavior and identifying churn drivers for a telecommunications company

---

## Notebook 02: Data Preparation
This notebook focuses on preparing the Telco Churn dataset for analysis. Key steps include handling missing values, standardizing feature names, and engineering meaningful variables. Two final DataFrames are produced: one optimized for business interpretation and another fully encoded for future machine learning applications.

---

## Table of Contents
- [2.0 Data Preparation](#20-data-preparation)
    - [2.1 Handling Missing Values](#21-handling-missing-values)
    - [2.2 Column Name Normalization](#22-column-name-normalization)
    - [2.3 Exploring Data (Feature Balance & Summaries)](#23-exploring-data-feature-balance--summaries)
        - [2.1 SQL Data Exploration](#231-sql-exploration)
    - [2.4 Encoding Categorical Variables](#24-encoding-categorical-variables)
        - [2.4.1 Setting DataFrame Indices](#241-setting-dataframe-indices)
        - [2.4.2 Encoding Binary Variables](#242-encoding-binary-values)
        - [2.4.3 Encoding Other Categorical Variables](#243-encoding-other-categorical-values)
    - [2.5 Discretizing Variables](#25-discretizing-variables)
        - [2.5.1 The Tenure Feature](#251-handling-the-tenure-feature)
        - [2.5.2 The Monthly Charges Feature](#252-handling-the-monthlycharges-feature)
        - [2.5.3 The Total Charges Feature](#253-handling-the-totalcharges-feature)
    - [2.6 Additional Feature Engineering](#26-additional-feature-engineering)
        - [2.6.1 Customer Tenure Features](#261-customer-tenure-features)
        - [2.6.2 Revenue Features](#262-revenue-features)
        - [2.6.3 Service Combination Features](#263-service-combination-features)
        - [2.6.4 Contract & Payment Features](#264-contract--payment-features)
        - [2.6.5 Interaction Features](#265-interaction-features)
    - [2.7 Summary](#27-summary)

## 2.0 Data Preparation <a class="anchor" id="20-data-preparation"></a>

Prior to exploratory data analysis and modeling, the data will have to be cleaned and processed to remove missing values, homogenize naming conventions, and ensure type accuracy. 

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [42]:
# Load the data
df = pd.read_csv("../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv")

## 2.1 Handling Missing Values <a class="anchor" id="21-handling-missing-values"></a>

Before performing any modeling or deeper analysis, it's important to ensure the dataset does not contain missing or invalid values. We've already identified missing `TotalCharges` values, but we should ensure that no other columns contain mising data.


In [43]:
# Force type change to numeric; convert empty strings to NaN values
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# Replace NaN values with 0
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [44]:
# Check missing values by percentage
missing = df.isna().mean().sort_values(ascending=False)
missing[missing > 0]

Series([], dtype: float64)

**Observation:**  
The only missing values were in `TotalCharges`, which correspond to new customers (tenure = 0). These have been set to 0 to reflect no charges billed yet.

## 2.2 Column Name Normalization <a class="anchor" id="22-column-name-normalization"></a>

The original dataset uses inconsistent casing in feature names
(e.g., `customerID`, `SeniorCitizen`, `gender`).
To ensure readability and consistency across SQL and Python,
all feature names were converted to lowercase `snake_case` format.

This improves maintainability without altering data semantics.

In [45]:
# Transform column headers to lowercase snake_case
df.columns = (
    df.columns
      .str.strip() # Remove leading/trailing spaces
      .str.replace(' ', '_') # Replace spaces with underscores
      .str.replace(r'(?<!^)(?=[A-Z][a-z])', '_', regex=True) # Insert underscores before single capital letters
      .str.replace(r'(?<!^)(?=[A-Z][A-Z])', '_', regex=True) # Insert underscores before double capital letters
      .str.lower()
)

## 2.3 Exploring Data (Feature Balance & Summaries) <a class="anchor" id="23-exploring-data-feature-balance--summaries"></a>
Binary label features should ideally be split evenly. Unbalanced label features may lead to label bias, causing models to overestimating or underestimating their predictive abilities - depending on which way it is biased. 

In [46]:
# Check churn feature balance
churn_counts = df['churn'].value_counts(normalize=True) * 100

# Check gender feature balance
gender_counts = df['gender'].value_counts(normalize=True) * 100

# Display feature balance
print(f'Churn distribution:\n\tNo: {churn_counts['No']:.1f}%\n\tYes: {churn_counts['Yes']:.1f}\n')
print(f'Gender distribution:\n\tMale: {gender_counts['Male']:.1f}%\n\tFemale: {gender_counts['Female']:.1f}%')

Churn distribution:
	No: 73.5%
	Yes: 26.5

Gender distribution:
	Male: 50.5%
	Female: 49.5%


**Observation:** The gender feature is fairly evenly split, with approximately 50.05% Males and 49.95% Females. The churn rate is uneven with 26.5% churned customers, and this imbalance will have to be accounted for in future modeling.

#### 2.3.1 SQL Exploration <a class="anchor" id="231-sql-exploration"></a>
To demonstrate SQL querying, we’ll use SQLite to explore customer churn distribution and summarize key categorical features. SQL is particularly useful for quick, repeatable aggregations and filters.

In [47]:
# Create connection to SQL database (in-memory)
conn = sqlite3.connect(':memory:')

# Convert DataFrame to SQL database 
df.to_sql('telco', conn, index=False)

7043

In [48]:
# Examine label column balance
query = """
    SELECT 
        contract,
        internet_service,
        COUNT(*) as total_customers,
        SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) as churned_customers,
        ROUND(AVG(CASE WHEN churn = 'Yes' THEN 1.0 ELSE 0 END) * 100, 2) AS churn_rate
    FROM telco
    GROUP BY contract, internet_service
    ORDER BY churn_rate DESC;
"""
pd.read_sql(query, conn)

,contract,internet_service,total_customers,churned_customers,churn_rate
0,Month-to-month,Fiber optic,2128,1162,54.61
1,Month-to-month,DSL,1223,394,32.22
2,One year,Fiber optic,539,104,19.29
3,Month-to-month,No,524,99,18.89
4,One year,DSL,570,53,9.30
5,Two year,Fiber optic,429,31,7.23
6,One year,No,364,9,2.47
7,Two year,DSL,628,12,1.91
8,Two year,No,638,5,0.78


## 2.4 Encoding Categorical Variables <a class="anchor" id="24-encoding-caegorical-variables"></a>
Many categorical features from the original dataset need to be encoded to enable certain data analysis and modeling techniques. Splitting these into one dataset geared towards business insights intelligibility, and one geared towards modeling will allow for the best of both worlds. 

In [49]:
# Create business insights DataFrame
df_readable = df.copy()
# Homogenize readable DataFrame Features
df_readable['senior_citizen'] = df['senior_citizen'].map({1: 'Yes', 0: 'No'})
df_readable.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#### 2.4.1 Setting DataFrame Indices <a class="anchor" id="241-setting-dataframe-indices"></a>
The customer_id column contains useful information for the encoded DataFrame, but is not numerical data, so it is best used as the index column for the encoded DataFrame. 

In [50]:
# Check that each customer_id is unique
# Each value is unique if the rows and the number of unique values are equal
len(df['customer_id'].unique()) == df.shape[0]

True

In [51]:
# Set the encoded df index column to match the customer_id
df.index = df['customer_id']
df.drop('customer_id', axis=1, inplace=True)
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
customer_id,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#### 2.4.2 Encoding Binary Values <a class="anchor" id="242-encoding-binary-values"></a>
The `Partner`, `Dependents`, `PhoneService`, `PaperlessBilling`, and `Churn` features are all binary 'Yes' or 'No' values; transforming these into binary 1 or 0 values will simplify working with these data. The gender feature can also be encoded in this way.

In [52]:
# Encode gender column
df['gender'] = df['gender'].map({'Female': 1, 'Male': 0})
# Rename column to reflect encoding change
df.rename(columns={'gender': 'gender_female'}, inplace=True)

# Encode binary 'Yes'/'No' columns
binary_cols = [col for col in df.columns if df[col].dtype == 'object' and df[col].nunique() == 2]
for col in binary_cols:
    df[col] = df[col].map({'Yes':1, 'No':0})

#### 2.4.3 Encoding Other Categorical Values <a class="anchor" id="243-encoding-other-categorical-values"></a>
The `multiple_lines`, `online_security`, `online_backup`, `device_protection`, `tech_support`, and `streaming_tv` features are all ternary, with 'Yes' or 'No' as well as a third column indicating a lack of internet or phone service. 

The `online_security`, `online_backup`, `device_protection`, `tech_support`, and `streaming_tv` features have a 'No internet service' value which perfectly corresponds to a 'No' value in the `internet_service` feature. This can be encoded as a separate feature.

In [53]:
# One-hot encoding for the remaining categorical features 
multi_cat_cols = [col for col in df.select_dtypes('object').columns if col not in binary_cols]
df = pd.get_dummies(df, columns=multi_cat_cols, drop_first=True) 
df.info() # Verify encoded types are correct

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 31 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gender_female                           7043 non-null   int64  
 1   senior_citizen                          7043 non-null   int64  
 2   partner                                 7043 non-null   int64  
 3   dependents                              7043 non-null   int64  
 4   tenure                                  7043 non-null   int64  
 5   phone_service                           7043 non-null   int64  
 6   paperless_billing                       7043 non-null   int64  
 7   monthly_charges                         7043 non-null   float64
 8   total_charges                           7043 non-null   float64
 9   churn                                   7043 non-null   int64  
 10  multiple_lines_No phone service         7043 non-n

## 2.5 Discretizing Variables <a class="anchor" id="25-discretizing-variables"></a>
The `tenure`, `MonthlyCharges`, and `TotalCharges` columns have a range of values which can be grouped into bins for the business insights-focused DataFrame. This makes visualization and interpretation easier, but risks losing some of the details and potentially introducing some bias into the dataset if done improperly. 

#### 2.5.1 Handling the Tenure Feature <a class="anchor" id="251-handling-the-tenure-feature"></a>

In [54]:
# Display where each year lands in the dataset's percentiles
for months in range(12, df_readable['tenure'].max() + 1, 12):
    print(f'{months} Month Percentile: {100 * np.sum(df_readable['tenure'] <= months)/df_readable.shape[0]:.0f}')

12 Month Percentile: 31
24 Month Percentile: 46
36 Month Percentile: 57
48 Month Percentile: 68
60 Month Percentile: 80
72 Month Percentile: 100


We can create three roughly equal-sized bins: 0-12 months (~31%), 13-48 months (~37%), and 49+ months (~32%). Splitting into equal width bins is better for statistical comparison and helps maintain balance in modeling, but is harder to interpret in business terms. 

We can create bins with an equal temporal range: 0-12, 13-24, 25-36, 37-48. Splitting into consistent time periods is intuitive and helpful for trend visualizations, but unequal sample sizes and small bins may hinder statistical modeling.


Alternatively, we could try to mix the two to maintain interpretability and bin-size balance:
- **New Customers (0-12 months):** The second-largest bin, with roughly 31% of customers using Telco's service for less than 1 year.

- **Established Customers (13-24 months):** Customers in a transition period between new and loyal customers; approximately 15% of customers fall in this bin, making it the smallest.

- **Mid-term Customers (25-48 months):** Customers who have been with the company for a reasonably long time, likely to be more stable; approximately 22% of customers fall into this bin.

- **Long-term Customers (49+ months):** Loyal customers who have been using Telco's service for over 4 years; the largest bin, with ~32% of customers.

In [55]:
bins = [0, 12, 24, 48, 72] # Create bins for each range of months
labels = ['0-12m', '13-24m', '25-48m', '49-72m']
# Create tenure_group column with tenure sorted into corresponding bins
df_readable['tenure_group'] = pd.cut(df_readable['tenure'], bins=bins, labels=labels, right=True, include_lowest=True)

#### 2.5.2 Handling the MonthlyCharges Feature <a class="anchor" id="252-handling-the-monthly-charges-feature"></a>

In [56]:
# Test potential monthly_charges bins
# Print how much of the dataset is contained by each $10 increment
for charges in range(10, int(df_readable['monthly_charges'].max()) + 5, 10):
    print(f'${charges} Monthly Charge Percentile: {100 * np.sum(df_readable['monthly_charges'] <= charges)/df.shape[0]:.0f}')

$10 Monthly Charge Percentile: 0
$20 Monthly Charge Percentile: 9
$30 Monthly Charge Percentile: 23
$40 Monthly Charge Percentile: 26
$50 Monthly Charge Percentile: 33
$60 Monthly Charge Percentile: 41
$70 Monthly Charge Percentile: 49
$80 Monthly Charge Percentile: 62
$90 Monthly Charge Percentile: 75
$100 Monthly Charge Percentile: 87
$110 Monthly Charge Percentile: 97
$120 Monthly Charge Percentile: 100


`MonthlyCharges` are fairly evenly distributed with a few outliers. It can be split into 5 segments of roughly equal size, with the first starting at $10, since the lowest value in the dataset is $18.25. Changes in future pricing will require adjusting these values to better discretize the data. 

In [57]:
bins = [10, 40, 70, 100, 120] # Create bins for each monthly spending bracket
labels = ['Low(<40$)' ,'Medium ($40-$70)', 'High ($70-$100)', 'Very High (>$100)']
# Create monthly_charge_group featurre with charges sorted into distinct bins
df_readable['monthly_charge_group'] = pd.cut(df_readable['monthly_charges'], bins=bins, labels=labels)

#### 2.5.3 Handling the TotalCharges Feature <a class="anchor" id="253-handling-the-totalcharges-feature"></a>

`TotalCharges` represents the cumulative revenue per customer and correlates strongly with both `tenure` and `MonthlyCharges`.  
We retain it as a continuous variable for quantitative analysis but also create business-defined value tiers to visualize churn across customer segments. The value tiers are split into 5 distinct groups, representing the bottom and top 10%, as well as the 25th, 50th, and 75th percentiles. These groups are suited to business analysis. 

This dual approach maintains analytical precision while improving interpretability for business stakeholders.


In [58]:
percentiles = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0] # Create bins for each percentile
bins = df['total_charges'].quantile(percentiles).values
labels = ['P0-10', 'P10-25', 'P25-50', 'P50-75', 'P75-90', 'P90-100']
# Create total_charge_group feature with total charges sorted by percentile
df_readable['total_charge_group'] = pd.cut(df_readable['total_charges'], bins=bins, labels=labels, include_lowest=True)

## 2.6 Additional Feature Engineering <a class="anchor" id="26-additional-feature-engineering"></a>

#### 2.6.1 Customer Tenure Features <a class="anchor" id="261-customer-tenure-features"></a>

We can measure tenure in years clarity in modeling, create a binary feature displaying whether a customer has been with Telco for less than 12 months (new customers), or whether they have been with Telco for more than 48 months (long-term customers). How long a customer has been with the company is often strongly correlated to churn, and encoding these features should aid future models. 

In [59]:
# Create tenure in years measure
df['tenure_years'] = df['tenure']/12

# Add new customer column
df['new_customer'] = df['tenure'].map(lambda x: 1 if x <= 12 else 0)

# Add long-term customer column
df['long_term_customer'] = df['tenure'].map(lambda x: 1 if x >= 48 else 0)

# Examine newly created columns
df[['tenure', 'tenure_years', 'new_customer', 'long_term_customer']]

,tenure,tenure_years,new_customer,long_term_customer
customer_id,,,,
7590-VHVEG,1,0.083333,1,0
5575-GNVDE,34,2.833333,0,0
3668-QPYBK,2,0.166667,1,0
7795-CFOCW,45,3.750000,0,0
9237-HQITU,2,0.166667,1,0
...,...,...,...,...
6840-RESVB,24,2.000000,0,0
2234-XADUH,72,6.000000,0,1
4801-JZAZL,11,0.916667,1,0


#### 2.6.2 Revenue Features <a class="anchor" id="262-revenue-features"></a>

We can better understand the customer revenue relationship through average monthly charges and revenue per year. This can help us quantify the revenues generated by each customer and appropriate investment levels to help prevent churn. 

In [60]:
# Calculate average monthly charges for each customer
df['avg_monthly_charges'] = df['total_charges'] / df['tenure']

# Calculate revenue per year by customer
df['revenue_per_year'] = df['total_charges'] / df['tenure_years']

#### 2.6.3 Service Combination Features <a class="anchor" id="263-service-combination-features"></a>

Customers who use more services from a telecommunications provider like Telco may "sticker" and less likely to churn. Having a count of the number of services each customer is using will help to identify possible correlations.

In [61]:
# Set has internet feature based on internet service column
df_readable['has_internet'] = df_readable['internet_service'].map({'DSL': 'Yes', 'Fiber optic': 'Yes', 'No': 'No'})

# Set redundant internet service values values to NaN
df_readable['internet_service'] = df_readable['internet_service'].map({'DSL': 'DSL', 'Fiber optic': 'Fiber optic', 'No': np.nan})

# Set redundant internet services features to NaN
internet_features = ['online_security', 'device_protection', 'online_backup', 'tech_support', 'streaming_tv', 'streaming_movies']
for feature in internet_features:
    df_readable[feature] = df_readable[feature].map({'Yes': 'Yes', 'No': 'No', 'No internet service': np.nan})

# Create summary has_streaming feature
df_readable['has_streaming'] = df_readable[[
    'streaming_tv', 
    'streaming_movies'
    ]].apply(lambda x: 'Yes' if (x.iloc[0] == 'Yes') or (x.iloc[1] == 'Yes') else 'No', axis=1)

# Set redundant multiple_lines values to Nan
df_readable['multiple_lines'] = df_readable['multiple_lines'].map({'Yes': 'Yes', 'No': 'No', 'No phone service': np.nan})

df_readable.tail(10)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,paperless_billing,payment_method,monthly_charges,total_charges,churn,tenure_group,monthly_charge_group,total_charge_group,has_internet,has_streaming
7033,9767-FFLEM,Male,No,No,No,38,Yes,No,Fiber optic,No,...,Yes,Credit card (automatic),69.50,2625.25,No,25-48m,Medium ($40-$70),P50-75,Yes,No
7034,0639-TSIQW,Female,No,No,No,67,Yes,Yes,Fiber optic,Yes,...,Yes,Credit card (automatic),102.95,6886.25,Yes,49-72m,Very High (>$100),P90-100,Yes,Yes
7035,8456-QDAVC,Male,No,No,No,19,Yes,No,Fiber optic,No,...,Yes,Bank transfer (automatic),78.70,1495.10,No,13-24m,High ($70-$100),P50-75,Yes,Yes
7036,7750-EYXWZ,Female,No,No,No,12,No,NaN,DSL,No,...,No,Electronic check,60.65,743.30,No,0-12m,Medium ($40-$70),P25-50,Yes,Yes
7037,2569-WGERO,Female,No,No,No,72,Yes,No,NaN,NaN,...,Yes,Bank transfer (automatic),21.15,1419.40,No,49-72m,Low(<40$),P50-75,No,No
7038,6840-RESVB,Male,No,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Mailed check,84.80,1990.50,No,13-24m,High ($70-$100),P50-75,Yes,Yes
7039,2234-XADUH,Female,No,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,Credit card (automatic),103.20,7362.90,No,49-72m,Very High (>$100),P90-100,Yes,Yes
7040,4801-JZAZL,Female,No,Yes,Yes,11,No,NaN,DSL,Yes,...,Yes,Electronic check,29.60,346.45,No,0-12m,Low(<40$),P10-25,Yes,No
7041,8361-LTMKD,Male,Yes,Yes,No,4,Yes,Yes,Fiber optic,No,...,Yes,Mailed check,74.40,306.60,Yes,0-12m,High ($70-$100),P10-25,Yes,No
7042,3186-AJIEK,Male,No,No,No,66,Yes,No,Fiber optic,Yes,...,Yes,Bank transfer (automatic),105.65,6844.50,No,49-72m,Very High (>$100),P90-100,Yes,Yes


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 36 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gender_female                           7043 non-null   int64  
 1   senior_citizen                          7043 non-null   int64  
 2   partner                                 7043 non-null   int64  
 3   dependents                              7043 non-null   int64  
 4   tenure                                  7043 non-null   int64  
 5   phone_service                           7043 non-null   int64  
 6   paperless_billing                       7043 non-null   int64  
 7   monthly_charges                         7043 non-null   float64
 8   total_charges                           7043 non-null   float64
 9   churn                                   7043 non-null   int64  
 10  multiple_lines_No phone service         7043 non-n

In [63]:
# Count number of services used for the encoded DataFrame

df['num_services'] = df[[
    'phone_service', 
    'online_security_Yes',
    'online_backup_Yes',
    'device_protection_Yes', 
    'tech_support_Yes', 
    'streaming_tv_Yes', 
    'streaming_movies_Yes'
]].apply(lambda x: (x == True).sum(), axis=1)

# Add internet service (logical inverse of internet_service_No)
df['num_services'] = df['num_services'] + ~df['internet_service_No']

# Check new features
df[['num_services', 'phone_service', 'online_security_Yes', 'tech_support_Yes', 'streaming_tv_Yes', 'streaming_movies_Yes']]

,num_services,phone_service,online_security_Yes,tech_support_Yes,streaming_tv_Yes,streaming_movies_Yes
customer_id,,,,,,
7590-VHVEG,2,0,False,False,False,False
5575-GNVDE,4,1,True,False,False,False
3668-QPYBK,4,1,True,False,False,False
7795-CFOCW,4,0,True,True,False,False
9237-HQITU,2,1,False,False,False,False
...,...,...,...,...,...,...
6840-RESVB,7,1,True,True,True,True
2234-XADUH,6,1,False,False,True,True
4801-JZAZL,2,0,True,False,False,False


In [64]:
# Count number of services used for the readable DataFrame
df_readable['num_services'] = df_readable[[
    'phone_service',
    'has_internet',
    'online_security', 
    'online_backup',
    'device_protection',
    'tech_support', 
    'streaming_tv', 
    'streaming_movies'
]].apply(lambda x: (x == 'Yes').sum(), axis=1)

# Check feature values
df_readable[['num_services', 'phone_service', 'online_security', 'tech_support', 'streaming_tv', 'streaming_movies']]

,num_services,phone_service,online_security,tech_support,streaming_tv,streaming_movies
0,2,No,No,No,No,No
1,4,Yes,Yes,No,No,No
2,4,Yes,Yes,No,No,No
3,4,No,Yes,Yes,No,No
4,2,Yes,No,No,No,No
...,...,...,...,...,...,...
7038,7,Yes,Yes,Yes,Yes,Yes
7039,6,Yes,No,No,Yes,Yes
7040,2,No,Yes,No,No,No
7041,2,Yes,No,No,No,No


Classifying customers by whether they have internet service or streaming will help in understanding which services are more closely correlated with churn. This will also simplify the internet-related feature values like online_security or tech_support.

#### 2.6.4 Contract & Payment Features <a class="anchor" id="264-contract--payment-features"></a>

The most transitory contract and payment features may be related to high churn rates, as they are easier to cancel and indicate less of a commitment. Pairing payment and contract features by commitment or time to set up may reveal new insights about churn rates. Indicating whether the payment method is automatic is another potential correlation to churn rates.

In [65]:
# Paperless payments and monthly 
df['paperless_and_monthly'] = df[[
    'payment_method_Credit card (automatic)', 
    'payment_method_Electronic check',
    'payment_method_Mailed check',
    'contract_One year',
    'contract_Two year'
]].apply(lambda x: (x == False).sum() == 0, axis=1)

# Check features
df[['paperless_and_monthly',
    'payment_method_Credit card (automatic)', 
    'payment_method_Electronic check',
    'payment_method_Mailed check',
    'contract_One year',
    'contract_Two year'
]].head()

,paperless_and_monthly,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,contract_One year,contract_Two year
customer_id,,,,,,
7590-VHVEG,False,False,True,False,False,False
5575-GNVDE,False,False,False,True,True,False
3668-QPYBK,False,False,False,True,False,False
7795-CFOCW,False,False,False,False,True,False
9237-HQITU,False,False,True,False,False,False


In [66]:
# # Automatic payments: True if automatic credit card payments or neither check method
df['automatic_payments'] = df[[
    'payment_method_Credit card (automatic)',
    'payment_method_Electronic check', 
    'payment_method_Mailed check'
]].apply(lambda x: (x.iloc[0] or not (x.iloc[1] or x.iloc[2])), axis=1) 

# Check newly created feature
df[['automatic_payments',
    'payment_method_Credit card (automatic)',
    'payment_method_Electronic check', 
    'payment_method_Mailed check'
]].head()

,automatic_payments,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check
customer_id,,,,
7590-VHVEG,False,False,True,False
5575-GNVDE,False,False,False,True
3668-QPYBK,False,False,False,True
7795-CFOCW,True,False,False,False
9237-HQITU,False,False,True,False


#### 2.6.5 Interaction Features <a class="anchor" id="265-interaction-features"></a>

The crux of data analysis is in the data relationships. By examining the charges per service we can judge whether customers are paying a premium per service, identifying potential consumer cost calculations influencing their decision to leave Telco. The most valuable customer segment is also woth examining, identified by having both high tenure and high monthly charge.

In [67]:
# Total charges per service 
df['total_charges_per_service'] = round(df['total_charges'] / (df['num_services'] + 1),2 )

# Total charges per service 
df['monthly_charges_per_service'] = round(df['monthly_charges'] / (df['num_services'] + 1),2 )

# Check new features
df[['total_charges_per_service', 'total_charges', 'monthly_charges', 'monthly_charges_per_service', 'num_services']].head()

,total_charges_per_service,total_charges,monthly_charges,monthly_charges_per_service,num_services
customer_id,,,,,
7590-VHVEG,9.95,29.85,29.85,9.95,2
5575-GNVDE,377.90,1889.50,56.95,11.39,4
3668-QPYBK,21.63,108.15,53.85,10.77,4
7795-CFOCW,368.15,1840.75,42.30,8.46,4
9237-HQITU,50.55,151.65,70.70,23.57,2


In [68]:
# Loyal high spenders (tenure greater than 48 months, monthly spending greater than $100)
df['loyal_high_spender'] = df[[
    'tenure', 
    'monthly_charges'
    ]].apply(lambda x: (x.iloc[0] >= 48 and x.iloc[1] >= 100), axis=1)

# Check features
df[['loyal_high_spender', 'tenure', 'monthly_charges']].tail()

,loyal_high_spender,tenure,monthly_charges
customer_id,,,
6840-RESVB,False,24,84.80
2234-XADUH,True,72,103.20
4801-JZAZL,False,11,29.60
8361-LTMKD,False,4,74.40
3186-AJIEK,True,66,105.65


Before proceeding to exploratory data analysis, we will review each of our now processed DataFrames, starting with the encoded DataFrame.

In [69]:
df.head()

,gender_female,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,...,new_customer,long_term_customer,avg_monthly_charges,revenue_per_year,num_services,paperless_and_monthly,automatic_payments,total_charges_per_service,monthly_charges_per_service,loyal_high_spender
customer_id,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,1,0,1,0,1,0,1,29.85,29.85,0,...,1,0,29.850000,358.200000,2,False,False,9.95,9.95,False
5575-GNVDE,0,0,0,0,34,1,0,56.95,1889.50,0,...,0,0,55.573529,666.882353,4,False,False,377.90,11.39,False
3668-QPYBK,0,0,0,0,2,1,1,53.85,108.15,1,...,1,0,54.075000,648.900000,4,False,False,21.63,10.77,False
7795-CFOCW,0,0,0,0,45,0,0,42.30,1840.75,0,...,0,0,40.905556,490.866667,4,False,True,368.15,8.46,False
9237-HQITU,1,0,0,0,2,1,1,70.70,151.65,1,...,1,0,75.825000,909.900000,2,False,False,50.55,23.57,False


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 42 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gender_female                           7043 non-null   int64  
 1   senior_citizen                          7043 non-null   int64  
 2   partner                                 7043 non-null   int64  
 3   dependents                              7043 non-null   int64  
 4   tenure                                  7043 non-null   int64  
 5   phone_service                           7043 non-null   int64  
 6   paperless_billing                       7043 non-null   int64  
 7   monthly_charges                         7043 non-null   float64
 8   total_charges                           7043 non-null   float64
 9   churn                                   7043 non-null   int64  
 10  multiple_lines_No phone service         7043 non-n

In [71]:
df_readable.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,payment_method,monthly_charges,total_charges,churn,tenure_group,monthly_charge_group,total_charge_group,has_internet,has_streaming,num_services
0,7590-VHVEG,Female,No,Yes,No,1,No,NaN,DSL,No,...,Electronic check,29.85,29.85,No,0-12m,Low(<40$),P0-10,Yes,No,2
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,...,Mailed check,56.95,1889.50,No,25-48m,Medium ($40-$70),P50-75,Yes,No,4
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,...,Mailed check,53.85,108.15,Yes,0-12m,Medium ($40-$70),P10-25,Yes,No,4
3,7795-CFOCW,Male,No,No,No,45,No,NaN,DSL,Yes,...,Bank transfer (automatic),42.30,1840.75,No,25-48m,Medium ($40-$70),P50-75,Yes,No,4
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,...,Electronic check,70.70,151.65,Yes,0-12m,High ($70-$100),P10-25,Yes,No,2


In [72]:
df_readable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   customer_id           7043 non-null   object  
 1   gender                7043 non-null   object  
 2   senior_citizen        7043 non-null   object  
 3   partner               7043 non-null   object  
 4   dependents            7043 non-null   object  
 5   tenure                7043 non-null   int64   
 6   phone_service         7043 non-null   object  
 7   multiple_lines        6361 non-null   object  
 8   internet_service      5517 non-null   object  
 9   online_security       5517 non-null   object  
 10  online_backup         5517 non-null   object  
 11  device_protection     5517 non-null   object  
 12  tech_support          5517 non-null   object  
 13  streaming_tv          5517 non-null   object  
 14  streaming_movies      5517 non-null   object  
 15  cont

**Observation:** These data appear to be correctly processed, so we may now save these data and move onto EDA.

In [73]:
# Save transformed data
df.to_csv('../data/processed/encoded_telco_churn_data.csv')
df_readable.to_csv('../data/processed/readable_telco_churn_data.csv', index=False)

- - - 
## 2.7 Summary <a class="anchor" id="27-summary"></a>

In this notebook, we prepared the Telco Churn dataset for analysis and future modeling by addressing missing values, standardizing feature names, and engineering new features. 
The only missing values were in `TotalCharges` for new customers, which were set to zero. Binary and ternary categorical variables were encoded to facilitate analysis and modeling, and continuous features such as `tenure`, `MonthlyCharges`, and `TotalCharges` were binned for business-focused interpretation while retaining their continuous form for quantitative analysis. 
Additional features were created to capture customer tenure, service usage, revenue contribution, and contract/payment characteristics, helping to quantify factors likely related to churn. 
With the dataset cleaned, standardized, and enriched with new features, it is now ready for exploratory analysis to uncover patterns and insights.

